In [1]:
print("ok")

ok


In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests

In [15]:
df = pd.read_csv(r"C:\Users\TPWODL\New folder_Content\TimeSeriesForecasting_End_To_End\data\raw\AEP_hourly.csv")

In [16]:
df

,Datetime,AEP_MW
0,2004-12-31 01:00:00,13478.0
1,2004-12-31 02:00:00,12865.0
2,2004-12-31 03:00:00,12577.0
3,2004-12-31 04:00:00,12517.0
4,2004-12-31 05:00:00,12670.0
...,...,...
121268,2018-01-01 20:00:00,21089.0
121269,2018-01-01 21:00:00,20999.0
121270,2018-01-01 22:00:00,20820.0
121271,2018-01-01 23:00:00,20415.0


In [17]:
df.tail()

,Datetime,AEP_MW
121268,2018-01-01 20:00:00,21089.0
121269,2018-01-01 21:00:00,20999.0
121270,2018-01-01 22:00:00,20820.0
121271,2018-01-01 23:00:00,20415.0
121272,2018-01-02 00:00:00,19993.0


In [13]:
df.shape

(121273, 2)

In [14]:
df.duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
121268    False
121269    False
121270    False
121271    False
121272    False
Length: 121273, dtype: bool

In [10]:
df.isnull().sum()

Datetime    0
AEP_MW      0
dtype: int64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121273 entries, 0 to 121272
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Datetime  121273 non-null  object 
 1   AEP_MW    121273 non-null  float64
dtypes: float64(1), object(1)
memory usage: 1.9+ MB


In [20]:

# ============================================
# 🧩 Step 1 — Create your time series DataFrame
# ============================================
data = {
    "Datetime": [
        "2018-01-01 20:00:00",
        "2018-01-01 21:00:00",
        "2018-01-01 22:00:00",
        "2018-01-01 23:00:00",
        "2018-01-02 00:00:00"
    ],
    "Load": [21089.0, 20999.0, 20820.0, 20415.0, 19993.0]
}

df = pd.DataFrame(data)
df["Datetime"] = pd.to_datetime(df["Datetime"])

# ============================================
# 🧠 Step 2 — Define your location coordinates
# (Example: Bhubaneswar, India)
# You can change these values if your region differs.
# ============================================
latitude = 20.2961
longitude = 85.8245

# ============================================
# 🌤 Step 3 — Fetch hourly temperature from Open-Meteo
# ============================================
start_date = df["Datetime"].min().strftime("%Y-%m-%d")
end_date   = df["Datetime"].max().strftime("%Y-%m-%d")

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": latitude,
    "longitude": longitude,
    "start_date": start_date,
    "end_date": end_date,
    "hourly": "temperature_2m",
    "timezone": "Asia/Kolkata"
}

response = requests.get(url, params=params)
weather = response.json()

# Convert API response to DataFrame
temp_df = pd.DataFrame({
    "Datetime": pd.to_datetime(weather["hourly"]["time"]),
    "Temperature": weather["hourly"]["temperature_2m"]
})

# ============================================
# 🔗 Step 4 — Merge with your load data
# ============================================
merged_df = pd.merge(df, temp_df, on="Datetime", how="left")

# ============================================
# ✅ Step 5 — Display final dataset
# ============================================
print(merged_df)


             Datetime     Load  Temperature
0 2018-01-01 20:00:00  21089.0         18.4
1 2018-01-01 21:00:00  20999.0         17.7
2 2018-01-01 22:00:00  20820.0         17.9
3 2018-01-01 23:00:00  20415.0         17.5
4 2018-01-02 00:00:00  19993.0         17.3
